# Setup

In [ ]:
# Parameters
videos_dir = None
scene_json_dirs = None
save_dir_all = None
metadata_dir_all = None
extract_metadata_only = None
# Change this to the desired number of frames per segment
num_frames_per_segment = None

In [ ]:
import os

dir_path = os.getcwd()


if not videos_dir:
    if 'google.colab' in str(get_ipython()):
        # Update this path as necessary
        videos_dir = f'{dir_path}/AIC_Video'
    elif 'kaggle' in str(get_ipython()):
        videos_dir = f'{dir_path}/AIC_Video'
    else:
        parent_dir_path = os.path.dirname(dir_path)
        videos_dir = f'{parent_dir_path}/dataset/AIC_Video'
    
if not scene_json_dirs:
    scene_json_dirs = f'{dir_path}/SceneJSON'
    
if not save_dir_all:
    save_dir_all = f'{dir_path}/Keyframes'
    
if not metadata_dir_all:
    metadata_dir_all = f'{dir_path}/Keyframes_Metadata'
    
if not extract_metadata_only:
    extract_metadata_only = False
    
if not num_frames_per_segment:
    num_frames_per_segment = 5  # Change this to the desired number of frames per segment

In [ ]:
!git clone https://github.com/soCzech/TransNetV2.git
!git lfs install
!cd TransNetV2
!git lfs fetch https://github.com/soCzech/TransNetV2.git
!git lfs checkout

fatal: destination path 'TransNetV2' already exists and is not an empty directory.


Updated Git hooks.
Git LFS initialized.
fetch: Fetching reference refs/heads/feat/app
Checking out LFS objects: 100% (4/4), 0 B | 0 B/s, done.                        


In [ ]:
import os
import json
import cv2
import numpy as np
from tqdm import tqdm

# Process cutting frames

In [ ]:
def numpy_to_python(obj):
    """Convert numpy types to standard Python types."""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj


def parse_video_info(videos_dir):
    """Parse video information from the directory structure."""
    all_video_paths = {}
    for part in sorted(os.listdir(videos_dir)):
        data_part = part.split('_')[-1]
        all_video_paths[data_part] = {}

    for data_part in sorted(all_video_paths.keys()):
        data_part_path = os.path.join(
            videos_dir, f'Videos_{data_part}', 'video')
        video_paths = sorted(os.listdir(data_part_path))
        video_ids = [video_path.replace('.mp4', '').split(
            '_')[-1] for video_path in video_paths]
        for video_id, video_path in zip(video_ids, video_paths):
            video_path_full = os.path.join(data_part_path, video_path)
            all_video_paths[data_part][video_id] = video_path_full

    return all_video_paths


def get_evenly_spaced_frames(start_idx, end_idx, num_frames):
    """Get evenly spaced frame indices between start_idx and end_idx."""
    return np.linspace(start_idx, end_idx, num_frames, dtype=int)


def get_frame_timestamp(frame_index, fps):
    """Convert frame index to timestamp in seconds."""
    return frame_index / fps


def create_directory(directory):
    """Create a directory if it doesn't exist."""
    if not os.path.exists(directory):
        os.makedirs(directory)


def get_relative_path(path, base_dir):
    """Get the relative path from base_dir to path."""
    return os.path.relpath(path, base_dir)


def get_video_fps(video_path):
    """Get the fps of a video file."""
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    return fps


def process_metadata(video_path, scene_json_path, save_dir, videos_dir, fps, num_frames_per_segment):
    """Process and extract metadata from video segments."""
    with open(scene_json_path, 'r') as f:
        video_scenes = json.load(f)

    frame_metadata = {}
    video_id = os.path.basename(video_path).split('.')[0]

    relative_video_path = get_relative_path(video_path, videos_dir)
    keyframes_dir = os.path.dirname(os.path.dirname(save_dir))

    for i, (start, end) in enumerate(video_scenes):
        frame_indices = get_evenly_spaced_frames(
            start, end, num_frames_per_segment)

        for index in frame_indices:
            timestamp = get_frame_timestamp(index, fps)
            filename = f"{index:06d}.jpg"
            filepath = os.path.join(save_dir, filename)
            relative_frame_path = get_relative_path(filepath, keyframes_dir)

            frame_metadata[f"{video_id}_{index:06d}"] = {
                "shot_index": numpy_to_python(i),
                "frame_index": numpy_to_python(index),
                "shot_start": numpy_to_python(start),
                "shot_end": numpy_to_python(end),
                "timestamp": numpy_to_python(timestamp),
                "video_path": relative_video_path,
                "frame_path": relative_frame_path
            }

    return frame_metadata


def sample_frames(video_path, scene_json_path, save_dir, videos_dir, fps, num_frames_per_segment):
    """Sample frames from video segments and save them to the specified directory."""
    create_directory(save_dir)

    frame_metadata = process_metadata(
        video_path, scene_json_path, save_dir, videos_dir, fps, num_frames_per_segment)

    cap = cv2.VideoCapture(video_path)

    for frame_id, metadata in frame_metadata.items():
        index = metadata['frame_index']
        cap.set(cv2.CAP_PROP_POS_FRAMES, index)
        ret, frame = cap.read()
        if ret:
            filename = f"{index:06d}.jpg"
            filepath = os.path.join(save_dir, filename)
            if not cv2.imwrite(filepath, frame):
                print('Failed to save frame:', filepath)
        else:
            print('Failed to read frame at index:', index)

    cap.release()
    return frame_metadata


def process_videos(all_video_paths, scene_json_dirs, save_dir_all, metadata_dir_all, videos_dir, num_frames_per_segment, extract_only=False):
    """Process all videos: sample frames (if needed) and extract metadata."""
    create_directory(save_dir_all)
    create_directory(metadata_dir_all)

    for key in all_video_paths.keys():
        save_dir = os.path.join(save_dir_all, key)

        if not all_video_paths[key]:
            print(f"Skipping empty AIC_Video subdirectory: {key}")
            continue

        scene_json_subdir = os.path.join(scene_json_dirs, key)
        if not os.path.exists(scene_json_subdir) or not os.listdir(scene_json_subdir):
            print(
                f"Skipping empty or non-existent SceneJSON subdirectory: {key}")
            continue

        create_directory(save_dir)

        video_paths_dict = all_video_paths[key]
        video_ids = sorted(video_paths_dict.keys())

        # Calculate fps once for this group of videos
        first_video_path = video_paths_dict[video_ids[0]]
        fps = get_video_fps(first_video_path)

        for video_id in tqdm(video_ids, desc=f"Processing videos in {key}"):
            video_path = video_paths_dict[video_id]
            video_scene_path = os.path.join(
                scene_json_dirs, key, f"{video_id}.json")

            if not os.path.exists(video_scene_path):
                print(f"Skipping missing SceneJSON file: {video_scene_path}")
                continue

            save_dir_video = os.path.join(save_dir, video_id)

            if extract_only:
                frame_metadata = process_metadata(
                    video_path, video_scene_path, save_dir_video, videos_dir, fps, num_frames_per_segment)
            else:
                frame_metadata = sample_frames(
                    video_path, video_scene_path, save_dir_video, videos_dir, fps, num_frames_per_segment)

            # Save metadata for this video
            metadata_filename = os.path.join(
                metadata_dir_all, f"{key}_{video_id}.json")
            with open(metadata_filename, 'w') as f:
                json.dump(frame_metadata, f, default=numpy_to_python)

In [ ]:
all_video_paths = parse_video_info(videos_dir)
process_videos(all_video_paths, scene_json_dirs, save_dir_all,
               metadata_dir_all, videos_dir, num_frames_per_segment, extract_only=False)

Processing videos in L01:   0%|          | 0/1 [00:00<?, ?it/s]

Processing videos in L01:   0%|          | 0/1 [00:23<?, ?it/s]


KeyboardInterrupt: 